# [Sec 0] submit a avg score csv
data set in trainging.csv
- id: ad identifier
- click: 0/1 for non-click/click
- hour: format is YYMMDDHH, so 14091123 means 23:00 on Sept. 11, 2014 UTC. *
- C1 – anonymized categorical variable *
- banner_pos
- site_id *
- site_domain *
- site_category *
- app_id *
- app_domain *
- app_category  *
- device_id
- device_ip *
- device_model
- device_type *
- device_conn_type *
- C14-C21 – anonymized categorical variables

In [1]:
import pandas as pd

In [2]:
sampleSubmission = pd.read_csv("data/sampleSubmission")

In [3]:
sampleSubmission.click = 0.16980562476404604
sampleSubmission.head()

,id,click
0,10000174058809263569,0.169806
1,10000182526920855428,0.169806
2,10000554139829213984,0.169806
3,10001094637809798845,0.169806
4,10001377041558670745,0.169806


In [4]:
# all avg-score can reach rank (1314/1604) 81 %
sampleSubmission.to_csv("avg_submission.csv",index=False)

# [Sec1] Train a xgb model with minimum features
```
pip install xgboost
```

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# training data is to large, sample ~0.5N
N= 40428967 
s = int(N/2) #desired sample size
skip = sorted(random.sample(range(1,N+1),N-s)) 
X_train = pd.read_csv("data/train", skiprows=skip)


In [3]:
# feat
from sklearn.feature_extraction import FeatureHasher
hasher = FeatureHasher(n_features=10,
            non_negative=True,
            input_type='string')

# create features

## site
tmp_features = hasher.transform(X_train.site_id.tolist()).toarray()
for i in range(10):
    X_train['site_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_train.site_domain.tolist()).toarray()
for i in range(10):
    X_train['site_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_train.site_category.tolist()).toarray()
for i in range(10):
    X_train['site_category'+str(i)]=tmp_features[:,i]

## app
tmp_features = hasher.transform(X_train.app_id.tolist()).toarray()
for i in range(10):
    X_train['app_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_train.app_domain.tolist()).toarray()
for i in range(10):
    X_train['app_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_train.app_category.tolist()).toarray()
for i in range(10):
    X_train['app_category'+str(i)]=tmp_features[:,i]

## devices
tmp_features = hasher.transform(X_train.device_id.tolist()).toarray()
for i in range(10):
    X_train['device_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_train.device_ip.tolist()).toarray()
for i in range(10):
    X_train['device_ip'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_train.device_model.tolist()).toarray()
for i in range(10):
    X_train['device_model'+str(i)]=tmp_features[:,i]

In [4]:
Site_list = ['site_id'+str(i) for i in range(10)]+['site_domain'+str(i) for i in range(10)]+ \
            ['site_category'+str(i) for i in range(10)]

App_list = ['app_id'+str(i) for i in range(10)]+['app_domain'+str(i) for i in range(10)]+ \
            ['app_category'+str(i) for i in range(10)]

Devices_list = ['device_id'+str(i) for i in range(10)]+['device_ip'+str(i) for i in range(10)]+ \
            ['device_model'+str(i) for i in range(10)]
X_train_trial = X_train[['banner_pos','C1','device_type','device_conn_type','C14','C15','C16','C17','C18',\
                         'C19','C20','C21']+Site_list+App_list+Devices_list]
y_target = X_train.click.values

In [16]:
X_train_trial.shape

(20214483, 102)

In [ ]:
import gc
gc.collect()

## [Sec1-1] train with xgboost

In [5]:
import xgboost as xgb

# make xgb dataset
msk = np.random.rand(len(X_train_trial)) < 0.8
dtrain = xgb.DMatrix(X_train_trial[msk], y_target[msk])
dtest = xgb.DMatrix(X_train_trial[~msk], y_target[~msk])

In [7]:
param = {'max_depth':8, 'eta':0.5, 'silent':1, 'objective':'binary:logistic' ,'subsample':0.3}
param['nthread'] = 6
param['eval_metric'] = 'logloss'
evallist  = [ (dtrain,'train'),(dtest,'eval')]
num_round = 8000
#bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=300)

[0]	train-logloss:0.51194	eval-logloss:0.511783
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 300 rounds.
[1]	train-logloss:0.452404	eval-logloss:0.452187
[2]	train-logloss:0.428944	eval-logloss:0.428672
[3]	train-logloss:0.419232	eval-logloss:0.41895
[4]	train-logloss:0.414807	eval-logloss:0.414517
[5]	train-logloss:0.412418	eval-logloss:0.412086
[6]	train-logloss:0.411184	eval-logloss:0.410877
[7]	train-logloss:0.410359	eval-logloss:0.410077
[8]	train-logloss:0.409864	eval-logloss:0.409585
[9]	train-logloss:0.409189	eval-logloss:0.408942
[10]	train-logloss:0.408961	eval-logloss:0.408715
[11]	train-logloss:0.408677	eval-logloss:0.408438
[12]	train-logloss:0.408337	eval-logloss:0.40811
[13]	train-logloss:0.407991	eval-logloss:0.407783
[14]	train-logloss:0.407703	eval-logloss:0.407498
[15]	train-logloss:0.407295	eval-logloss:0.407111
[16]	train-logloss:0.407261	eval-logloss:0.407085
[17]	train-lo

KeyboardInterrupt: 

## [Sec1-1] train with lightgbm
xgboost is too slowly and memory require much.

In [ ]:
import lightgbm as lgb

In [ ]:
#create lightgbm dataset
msk = np.random.rand(len(X_train_trial)) < 0.8
lgb_train = lgb.Dataset(X_train_trial[msk], y_target[msk])
lgb_eval = lgb.Dataset(X_train_trial[~msk], y_target[~msk],reference=lgb_train)


In [18]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': { 'binary_logloss'},
    'num_leaves': 31, # defauly leaves(31) amount for each tree
    'learning_rate': 0.08,
    'feature_fraction': 0.7, # will select 70% features before training each tree
    'bagging_fraction': 0.3, #feature_fraction, but this will random select part of data
    'bagging_freq': 5, #  perform bagging at every 5 iteration
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=30000,
                valid_sets=lgb_eval,
                early_stopping_rounds=500)

Start training...
[1]	valid_0's binary_logloss: 0.669921
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's binary_logloss: 0.648873
[3]	valid_0's binary_logloss: 0.629784
[4]	valid_0's binary_logloss: 0.61245
[5]	valid_0's binary_logloss: 0.596733
[6]	valid_0's binary_logloss: 0.58235
[7]	valid_0's binary_logloss: 0.569253
[8]	valid_0's binary_logloss: 0.557188
[9]	valid_0's binary_logloss: 0.546069
[10]	valid_0's binary_logloss: 0.535833
[11]	valid_0's binary_logloss: 0.526474
[12]	valid_0's binary_logloss: 0.517849
[13]	valid_0's binary_logloss: 0.509922
[14]	valid_0's binary_logloss: 0.502673
[15]	valid_0's binary_logloss: 0.495999
[16]	valid_0's binary_logloss: 0.489753
[17]	valid_0's binary_logloss: 0.484038
[18]	valid_0's binary_logloss: 0.478713
[19]	valid_0's binary_logloss: 0.47386
[20]	valid_0's binary_logloss: 0.469357
[21]	valid_0's binary_logloss: 0.4652
[22]	valid_0's binary_logloss: 0.461217
[23]	valid_0's binary_logloss: 0.457645
[24]	valid_0'

[202]	valid_0's binary_logloss: 0.403508
[203]	valid_0's binary_logloss: 0.40347
[204]	valid_0's binary_logloss: 0.403442
[205]	valid_0's binary_logloss: 0.40341
[206]	valid_0's binary_logloss: 0.403381
[207]	valid_0's binary_logloss: 0.403343
[208]	valid_0's binary_logloss: 0.403323
[209]	valid_0's binary_logloss: 0.403298
[210]	valid_0's binary_logloss: 0.403259
[211]	valid_0's binary_logloss: 0.403228
[212]	valid_0's binary_logloss: 0.403202
[213]	valid_0's binary_logloss: 0.403182
[214]	valid_0's binary_logloss: 0.403154
[215]	valid_0's binary_logloss: 0.403124
[216]	valid_0's binary_logloss: 0.403097
[217]	valid_0's binary_logloss: 0.403073
[218]	valid_0's binary_logloss: 0.403032
[219]	valid_0's binary_logloss: 0.403009
[220]	valid_0's binary_logloss: 0.402981
[221]	valid_0's binary_logloss: 0.402949
[222]	valid_0's binary_logloss: 0.40293
[223]	valid_0's binary_logloss: 0.402906
[224]	valid_0's binary_logloss: 0.402881
[225]	valid_0's binary_logloss: 0.402851
[226]	valid_0's bin

[403]	valid_0's binary_logloss: 0.399847
[404]	valid_0's binary_logloss: 0.399833
[405]	valid_0's binary_logloss: 0.399821
[406]	valid_0's binary_logloss: 0.399815
[407]	valid_0's binary_logloss: 0.399807
[408]	valid_0's binary_logloss: 0.399802
[409]	valid_0's binary_logloss: 0.399796
[410]	valid_0's binary_logloss: 0.399791
[411]	valid_0's binary_logloss: 0.399784
[412]	valid_0's binary_logloss: 0.399772
[413]	valid_0's binary_logloss: 0.399762
[414]	valid_0's binary_logloss: 0.399755
[415]	valid_0's binary_logloss: 0.399746
[416]	valid_0's binary_logloss: 0.399735
[417]	valid_0's binary_logloss: 0.399728
[418]	valid_0's binary_logloss: 0.399712
[419]	valid_0's binary_logloss: 0.399699
[420]	valid_0's binary_logloss: 0.399686
[421]	valid_0's binary_logloss: 0.399673
[422]	valid_0's binary_logloss: 0.399664
[423]	valid_0's binary_logloss: 0.399658
[424]	valid_0's binary_logloss: 0.399649
[425]	valid_0's binary_logloss: 0.399642
[426]	valid_0's binary_logloss: 0.399622
[427]	valid_0's 

[604]	valid_0's binary_logloss: 0.398197
[605]	valid_0's binary_logloss: 0.398193
[606]	valid_0's binary_logloss: 0.398177
[607]	valid_0's binary_logloss: 0.398171
[608]	valid_0's binary_logloss: 0.398156
[609]	valid_0's binary_logloss: 0.398142
[610]	valid_0's binary_logloss: 0.398137
[611]	valid_0's binary_logloss: 0.398132
[612]	valid_0's binary_logloss: 0.398122
[613]	valid_0's binary_logloss: 0.398115
[614]	valid_0's binary_logloss: 0.398108
[615]	valid_0's binary_logloss: 0.398104
[616]	valid_0's binary_logloss: 0.398101
[617]	valid_0's binary_logloss: 0.398097
[618]	valid_0's binary_logloss: 0.398091
[619]	valid_0's binary_logloss: 0.398088
[620]	valid_0's binary_logloss: 0.398079
[621]	valid_0's binary_logloss: 0.398073
[622]	valid_0's binary_logloss: 0.39807
[623]	valid_0's binary_logloss: 0.398063
[624]	valid_0's binary_logloss: 0.398061
[625]	valid_0's binary_logloss: 0.398055
[626]	valid_0's binary_logloss: 0.398052
[627]	valid_0's binary_logloss: 0.398043
[628]	valid_0's b

[805]	valid_0's binary_logloss: 0.397101
[806]	valid_0's binary_logloss: 0.397097
[807]	valid_0's binary_logloss: 0.397089
[808]	valid_0's binary_logloss: 0.397083
[809]	valid_0's binary_logloss: 0.397073
[810]	valid_0's binary_logloss: 0.397069
[811]	valid_0's binary_logloss: 0.397066
[812]	valid_0's binary_logloss: 0.397062
[813]	valid_0's binary_logloss: 0.397059
[814]	valid_0's binary_logloss: 0.397052
[815]	valid_0's binary_logloss: 0.397045
[816]	valid_0's binary_logloss: 0.397043
[817]	valid_0's binary_logloss: 0.39704
[818]	valid_0's binary_logloss: 0.397039
[819]	valid_0's binary_logloss: 0.397035
[820]	valid_0's binary_logloss: 0.397033
[821]	valid_0's binary_logloss: 0.397027
[822]	valid_0's binary_logloss: 0.397014
[823]	valid_0's binary_logloss: 0.397009
[824]	valid_0's binary_logloss: 0.397003
[825]	valid_0's binary_logloss: 0.396999
[826]	valid_0's binary_logloss: 0.396992
[827]	valid_0's binary_logloss: 0.396989
[828]	valid_0's binary_logloss: 0.396984
[829]	valid_0's b

...

[14045]	valid_0's binary_logloss: 0.390679
[14046]	valid_0's binary_logloss: 0.390679
[14047]	valid_0's binary_logloss: 0.390679
[14048]	valid_0's binary_logloss: 0.390679
[14049]	valid_0's binary_logloss: 0.390679
[14050]	valid_0's binary_logloss: 0.390679
[14051]	valid_0's binary_logloss: 0.390678
[14052]	valid_0's binary_logloss: 0.390678
[14053]	valid_0's binary_logloss: 0.390679
[14054]	valid_0's binary_logloss: 0.390679
[14055]	valid_0's binary_logloss: 0.390678
[14056]	valid_0's binary_logloss: 0.390678
[14057]	valid_0's binary_logloss: 0.390678
[14058]	valid_0's binary_logloss: 0.390678
[14059]	valid_0's binary_logloss: 0.390678
[14060]	valid_0's binary_logloss: 0.390679
[14061]	valid_0's binary_logloss: 0.390678
[14062]	valid_0's binary_logloss: 0.390678
[14063]	valid_0's binary_logloss: 0.390678
[14064]	valid_0's binary_logloss: 0.390678
[14065]	valid_0's binary_logloss: 0.390678
[14066]	valid_0's binary_logloss: 0.390678
[14067]	valid_0's binary_logloss: 0.390678
[14068]	val

[14236]	valid_0's binary_logloss: 0.390663
[14237]	valid_0's binary_logloss: 0.390662
[14238]	valid_0's binary_logloss: 0.390662
[14239]	valid_0's binary_logloss: 0.390662
[14240]	valid_0's binary_logloss: 0.390661
[14241]	valid_0's binary_logloss: 0.390662
[14242]	valid_0's binary_logloss: 0.390661
[14243]	valid_0's binary_logloss: 0.390661
[14244]	valid_0's binary_logloss: 0.390661
[14245]	valid_0's binary_logloss: 0.390661
[14246]	valid_0's binary_logloss: 0.39066
[14247]	valid_0's binary_logloss: 0.39066
[14248]	valid_0's binary_logloss: 0.39066
[14249]	valid_0's binary_logloss: 0.39066
[14250]	valid_0's binary_logloss: 0.39066
[14251]	valid_0's binary_logloss: 0.39066
[14252]	valid_0's binary_logloss: 0.39066
[14253]	valid_0's binary_logloss: 0.39066
[14254]	valid_0's binary_logloss: 0.39066
[14255]	valid_0's binary_logloss: 0.390659
[14256]	valid_0's binary_logloss: 0.39066
[14257]	valid_0's binary_logloss: 0.39066
[14258]	valid_0's binary_logloss: 0.39066
[14259]	valid_0's binar

[14428]	valid_0's binary_logloss: 0.390643
[14429]	valid_0's binary_logloss: 0.390643
[14430]	valid_0's binary_logloss: 0.390643
[14431]	valid_0's binary_logloss: 0.390643
[14432]	valid_0's binary_logloss: 0.390643
[14433]	valid_0's binary_logloss: 0.390643
[14434]	valid_0's binary_logloss: 0.390643
[14435]	valid_0's binary_logloss: 0.390643
[14436]	valid_0's binary_logloss: 0.390643
[14437]	valid_0's binary_logloss: 0.390643
[14438]	valid_0's binary_logloss: 0.390643
[14439]	valid_0's binary_logloss: 0.390643
[14440]	valid_0's binary_logloss: 0.390642
[14441]	valid_0's binary_logloss: 0.390642
[14442]	valid_0's binary_logloss: 0.390641
[14443]	valid_0's binary_logloss: 0.390642
[14444]	valid_0's binary_logloss: 0.390641
[14445]	valid_0's binary_logloss: 0.390641
[14446]	valid_0's binary_logloss: 0.390641
[14447]	valid_0's binary_logloss: 0.390641
[14448]	valid_0's binary_logloss: 0.390641
[14449]	valid_0's binary_logloss: 0.390641
[14450]	valid_0's binary_logloss: 0.390641
[14451]	val

[14620]	valid_0's binary_logloss: 0.390619
[14621]	valid_0's binary_logloss: 0.390619
[14622]	valid_0's binary_logloss: 0.390619
[14623]	valid_0's binary_logloss: 0.390618
[14624]	valid_0's binary_logloss: 0.390618
[14625]	valid_0's binary_logloss: 0.390618
[14626]	valid_0's binary_logloss: 0.390618
[14627]	valid_0's binary_logloss: 0.390618
[14628]	valid_0's binary_logloss: 0.390617
[14629]	valid_0's binary_logloss: 0.390617
[14630]	valid_0's binary_logloss: 0.390617
[14631]	valid_0's binary_logloss: 0.390617
[14632]	valid_0's binary_logloss: 0.390617
[14633]	valid_0's binary_logloss: 0.390617
[14634]	valid_0's binary_logloss: 0.390616
[14635]	valid_0's binary_logloss: 0.390616
[14636]	valid_0's binary_logloss: 0.390616
[14637]	valid_0's binary_logloss: 0.390616
[14638]	valid_0's binary_logloss: 0.390616
[14639]	valid_0's binary_logloss: 0.390616
[14640]	valid_0's binary_logloss: 0.390616
[14641]	valid_0's binary_logloss: 0.390616
[14642]	valid_0's binary_logloss: 0.390616
[14643]	val

[14812]	valid_0's binary_logloss: 0.390599
[14813]	valid_0's binary_logloss: 0.390599
[14814]	valid_0's binary_logloss: 0.390599
[14815]	valid_0's binary_logloss: 0.390599
[14816]	valid_0's binary_logloss: 0.390599
[14817]	valid_0's binary_logloss: 0.390599
[14818]	valid_0's binary_logloss: 0.390599
[14819]	valid_0's binary_logloss: 0.390599
[14820]	valid_0's binary_logloss: 0.390599
[14821]	valid_0's binary_logloss: 0.390599
[14822]	valid_0's binary_logloss: 0.390599
[14823]	valid_0's binary_logloss: 0.390599
[14824]	valid_0's binary_logloss: 0.390599
[14825]	valid_0's binary_logloss: 0.390599
[14826]	valid_0's binary_logloss: 0.390599
[14827]	valid_0's binary_logloss: 0.390599
[14828]	valid_0's binary_logloss: 0.390599
[14829]	valid_0's binary_logloss: 0.390599
[14830]	valid_0's binary_logloss: 0.390599
[14831]	valid_0's binary_logloss: 0.390599
[14832]	valid_0's binary_logloss: 0.390599
[14833]	valid_0's binary_logloss: 0.390599
[14834]	valid_0's binary_logloss: 0.390599
[14835]	val

In [26]:
gbm.best_score

defaultdict(dict, {'valid_0': {'binary_logloss': 0.3905854342103155}})

In [35]:
y_hat = gbm.predict(X_train_trial[:5])
print(y_hat)

[ 0.26516502  0.15455678  0.02903816  0.1106695   0.25826203]


In [19]:
gbm.save_model('lightgbm_20170718.txt')

In [20]:

#restore model
bst = lgb.Booster(model_file='lightgbm_20170718.txt')


In [34]:
y_hat = bst.predict(X_train_trial[:5])

[ 0.26516502  0.15455678  0.02903816  0.1106695   0.25826203]


# [Sec 2] predict the testing set , and submission

In [1]:
# since the memory limmit , we restart our python process
import pandas as pd
import numpy as np
import random

In [2]:
X_test = pd.read_csv("data/test")

In [3]:
sampleSubmission = pd.read_csv("data/sampleSubmission")

In [6]:
(X_test.id == sampleSubmission.id).all()

True

In [7]:
# create testing features
from sklearn.feature_extraction import FeatureHasher
hasher = FeatureHasher(n_features=10,
            non_negative=True,
            input_type='string')

# create features

## site
tmp_features = hasher.transform(X_test.site_id.tolist()).toarray()
for i in range(10):
    X_test['site_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_test.site_domain.tolist()).toarray()
for i in range(10):
    X_test['site_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_test.site_category.tolist()).toarray()
for i in range(10):
    X_test['site_category'+str(i)]=tmp_features[:,i]

## app
tmp_features = hasher.transform(X_test.app_id.tolist()).toarray()
for i in range(10):
    X_test['app_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_test.app_domain.tolist()).toarray()
for i in range(10):
    X_test['app_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_test.app_category.tolist()).toarray()
for i in range(10):
    X_test['app_category'+str(i)]=tmp_features[:,i]

## devices
tmp_features = hasher.transform(X_test.device_id.tolist()).toarray()
for i in range(10):
    X_test['device_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_test.device_ip.tolist()).toarray()
for i in range(10):
    X_test['device_ip'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_test.device_model.tolist()).toarray()
for i in range(10):
    X_test['device_model'+str(i)]=tmp_features[:,i]

In [8]:
Site_list = ['site_id'+str(i) for i in range(10)]+['site_domain'+str(i) for i in range(10)]+ \
            ['site_category'+str(i) for i in range(10)]

App_list = ['app_id'+str(i) for i in range(10)]+['app_domain'+str(i) for i in range(10)]+ \
            ['app_category'+str(i) for i in range(10)]

Devices_list = ['device_id'+str(i) for i in range(10)]+['device_ip'+str(i) for i in range(10)]+ \
            ['device_model'+str(i) for i in range(10)]
X_test_trial = X_test[['banner_pos','C1','device_type','device_conn_type','C14','C15','C16','C17','C18',\
                         'C19','C20','C21']+Site_list+App_list+Devices_list]


In [9]:
X_test_trial.shape

(4577464, 102)

In [10]:
X_train = []
X_test = []
import gc
gc.collect()

2423

In [12]:
import lightgbm as lgb
#restore model
bst = lgb.Booster(model_file='lightgbm_20170718.txt')


In [13]:
y_hat = bst.predict(X_test_trial)

In [15]:
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("naive_feature_hashing.csv",index=False)